In [1]:
import sys
sys.path.append('../')
import fm
import numpy as np
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.fx import subgraph_rewriter, symbolic_trace
import utils
from torch.fx import Proxy, Graph, GraphModule
from torch.fx.passes.utils.matcher_utils import SubgraphMatcher

replace success!


In [2]:
def gen_pattern_replace_and_matcher_for_FM(traced,
                                                  redency_part_slice,unredency_part_slice,
                                                  target_node_name,match_func = None
                                                ):
  from torch.fx.passes.utils.matcher_utils import SubgraphMatcher

  class PatternClass(torch.nn.Module):
    def __init__(self):
      super().__init__()
      self.embed = torch.nn.Embedding(1, 1)
      # self.offsets = torch.as_tensor(np.array((0, *np.cumsum([10,10])[:-1]), dtype=np.int64))
      self.offsets = nn.Parameter(torch.as_tensor(np.array((0, *np.cumsum([10,10])[:-1]), dtype=np.int64)),False)
    def forward(self,x):
      x = x + self.offsets
      x = self.embed(x)
      square_of_sum = torch.sum(x, dim=1) ** 2
      sum_of_square = torch.sum(x ** 2, dim=1)
      ix = square_of_sum - sum_of_square    
      ix = torch.sum(ix, dim=1, keepdim=True)  
      return 0.5 * ix
  def _match(match,ori,pat):
    return True 
  # env  = utils.get_env(traced)
  # target_node = env[target_node_name]
  # target_node_mod = utils.get_target_mod(traced,target_node_name,"_")
  pattern = PatternClass()  
  pattern_trace = symbolic_trace(pattern)
  pattern_graph = pattern_trace.graph
  original_graph = traced.graph
  matcher =  SubgraphMatcher(pattern_graph, match_output=False, match_placeholder=False,
                              remove_overlapping_matches=True)
  _matches = matcher.match(original_graph)
  match_filters = [_match if match_func is None else match_func]
  _matches = [
      m for m in _matches
      if all(match_filter(m, original_graph, pattern_graph)
              for match_filter in match_filters)
  ]  
  # 因为在过滤器中做了限制应该只有一个符合要求的
  _matched = _matches[0]
  pattern_env = utils.get_env(pattern_trace)
  node_map = _matched.nodes_map
  pn = pattern_env['offsets']
  offsets_node = node_map[pn]
  offsets_val = utils.get_target_mod(traced,offsets_node.target)
  
  embed_node = node_map[pattern_env['embed']]
  embed_node_module = utils.get_target_mod(traced,embed_node.target)
  

  
  
  class ReplacementClass(torch.nn.Module):
    def __init__(self):
      super().__init__()
      self.redency_offset = nn.Parameter(torch.as_tensor(np.array(offsets_val[redency_part_slice[1]], dtype=np.int64)),False)
      self.unredency_offset = nn.Parameter(torch.as_tensor(np.array(offsets_val[unredency_part_slice[1]], dtype=np.int64)),False)
      embedding_config = embed_node_module.weight.data.shape
      self.embed = nn.Embedding(embedding_config[0],embedding_config[1])
      self.embed.weight.data.copy_(embed_node_module.weight.data)
    def forward(self,x):
      redency_part = x[redency_part_slice] + self.redency_offset
      unredency_part = x[unredency_part_slice] + self.unredency_offset
      redency_embed = self.embed(redency_part)
      unredency_embed = self.embed(unredency_part)
      redency_embed_sum = torch.sum(redency_embed,dim=0)
      unredency_embed_sum = torch.sum(unredency_embed,dim=1)
      square_of_sum = (redency_embed_sum + unredency_embed_sum) ** 2
      redency_embed_square_sum = torch.sum(redency_embed ** 2,dim=0)
      unredency_embed_square_sum = torch.sum(unredency_embed ** 2,dim=1)
      sum_of_square = redency_embed_square_sum + unredency_embed_square_sum
      ix = square_of_sum - sum_of_square
      ix = torch.sum(ix,dim = 1,keepdim=True)
      return 0.5 * ix
    
  
  return pattern,ReplacementClass(),_match

In [3]:
def workload_fm(dim,redundancy):
  print(f"now gen workload of fm with config: dim: {dim}")
  fm_model_ori = fm.FactorizationMachineModel([100 for i in range(100)],dim)
  fm_model_traced_ori = symbolic_trace(fm_model_ori)
  
  fm_model_modify = fm.FactorizationMachineModel([100 for i in range(100)],dim)
  fm_model_traced_modify = symbolic_trace(fm_model_modify)
  pattern,replace,match = gen_pattern_replace_and_matcher_for_FM(fm_model_traced_modify,
                                                                      (0,slice(None,redundancy,None)),(slice(None,None,None),slice(redundancy,None,None)),
                                                                      "linear_offsets")
  matches = subgraph_rewriter.replace_pattern_with_filters(fm_model_traced_modify, pattern, replace,[match])
  return fm_model_traced_ori,fm_model_traced_modify

In [4]:
ori , modify = workload_fm(128,80)

now gen workload of fm with config: dim: 128


In [5]:
%timeit -n 1 -r 30 with torch.no_grad():ori(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))

67.8 ms ± 4.12 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [6]:
%timeit -n 1 -r 30 with torch.no_grad():modify(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))

22 ms ± 1.71 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [7]:
ori , modify = workload_fm(128,60)

now gen workload of fm with config: dim: 128


In [9]:
%timeit -n 1 -r 30 with torch.no_grad():modify(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))

34.2 ms ± 2.86 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [10]:
ori , modify = workload_fm(128,40)

now gen workload of fm with config: dim: 128


In [11]:
%timeit -n 1 -r 30 with torch.no_grad():modify(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))

44.4 ms ± 2.1 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [12]:
ori , modify = workload_fm(128,20)

now gen workload of fm with config: dim: 128


In [13]:
%timeit -n 1 -r 30 with torch.no_grad():modify(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))

56.5 ms ± 2.94 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [10]:
ori.print_readable()

class FactorizationMachineModel(torch.nn.Module):
    def forward(self, x):
        # No stacktrace found for following nodes
        linear_offsets = self.linear.offsets
        add = x + linear_offsets;  linear_offsets = None
        linear_fc = self.linear.fc(add);  add = None
        sum_1 = torch.sum(linear_fc, dim = 1);  linear_fc = None
        linear_bias = self.linear.bias
        add_1 = sum_1 + linear_bias;  sum_1 = linear_bias = None
        embedding_offsets = self.embedding.offsets
        add_2 = x + embedding_offsets;  x = embedding_offsets = None
        embedding_embedding = self.embedding.embedding(add_2);  add_2 = None
        sum_2 = torch.sum(embedding_embedding, dim = 1)
        pow_1 = sum_2 ** 2;  sum_2 = None
        pow_2 = embedding_embedding ** 2;  embedding_embedding = None
        sum_3 = torch.sum(pow_2, dim = 1);  pow_2 = None
        sub = pow_1 - sum_3;  pow_1 = sum_3 = None
        sum_4 = torch.sum(sub, dim = 1, keepdim = True);  sub = None
        

'class FactorizationMachineModel(torch.nn.Module):\n    def forward(self, x):\n        # No stacktrace found for following nodes\n        linear_offsets = self.linear.offsets\n        add = x + linear_offsets;  linear_offsets = None\n        linear_fc = self.linear.fc(add);  add = None\n        sum_1 = torch.sum(linear_fc, dim = 1);  linear_fc = None\n        linear_bias = self.linear.bias\n        add_1 = sum_1 + linear_bias;  sum_1 = linear_bias = None\n        embedding_offsets = self.embedding.offsets\n        add_2 = x + embedding_offsets;  x = embedding_offsets = None\n        embedding_embedding = self.embedding.embedding(add_2);  add_2 = None\n        sum_2 = torch.sum(embedding_embedding, dim = 1)\n        pow_1 = sum_2 ** 2;  sum_2 = None\n        pow_2 = embedding_embedding ** 2;  embedding_embedding = None\n        sum_3 = torch.sum(pow_2, dim = 1);  pow_2 = None\n        sub = pow_1 - sum_3;  pow_1 = sum_3 = None\n        sum_4 = torch.sum(sub, dim = 1, keepdim = True);  s

In [9]:
interp = utils.ProfilingInterpreter(ori)
interp.run(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))
print(interp.summary(True))

total true time 38.69748115539551 ms
total time: 51.5139102935791 ms
Op type        Op                     Average runtime (ms)    Pct total runtime
-------------  -------------------  ----------------------  -------------------
call_module    embedding_embedding              16.9585              32.9202
call_function  pow_2                            12.4335              24.1363
call_function  sum_2                             3.04627              5.9135
call_function  sum_3                             2.63882              5.12253
call_function  add                               0.842333             1.63516
call_module    linear_fc                         0.714302             1.38662
call_function  add_2                             0.558615             1.0844
call_function  sub                               0.513315             0.996459
call_function  pow_1                             0.434399             0.843265
call_function  sum_1                             0.119925             0

In [11]:
def gen_pattern_replace_and_matcher_for_FM_pow(traced,
                                                  redency_part_slice,unredency_part_slice,
                                                  target_node_name,batch = 4096,match_func = None
                                                ):
  from torch.fx.passes.utils.matcher_utils import SubgraphMatcher

  class PatternClass(torch.nn.Module):
    def __init__(self):
      super().__init__()
      self.embed = torch.nn.Embedding(1, 1)
      self.offsets = torch.as_tensor(np.array((0, *np.cumsum([10,10])[:-1]), dtype=np.int64))

    def forward(self,x):
      x = x + self.offsets
      x = self.embed(x)
      square_of_sum = torch.sum(x, dim=1) ** 2
      sum_of_square = torch.sum(x ** 2, dim=1)
      ix = square_of_sum - sum_of_square    
      ix = torch.sum(ix, dim=1, keepdim=True)  
      return 0.5 * ix
  def _match(match,ori,pat):
    return True 
  # env  = utils.get_env(traced)
  # target_node = env[target_node_name]
  # target_node_mod = utils.get_target_mod(traced,target_node_name,"_")
  pattern = PatternClass()  
  pattern_trace = symbolic_trace(pattern)
  pattern_graph = pattern_trace.graph
  original_graph = traced.graph
  matcher =  SubgraphMatcher(pattern_graph, match_output=False, match_placeholder=False,
                              remove_overlapping_matches=True)
  _matches = matcher.match(original_graph)
  match_filters = [_match if match_func is None else match_func]
  _matches = [
      m for m in _matches
      if all(match_filter(m, original_graph, pattern_graph)
              for match_filter in match_filters)
  ]  
  # 因为在过滤器中做了限制应该只有一个符合要求的
  _matched = _matches[0]
  pattern_env = utils.get_env(pattern_trace)
  node_map = _matched.nodes_map
  pn = pattern_env['offsets']
  offsets_node = node_map[pn]
  offsets_val = utils.get_target_mod(traced,offsets_node.target)
  
  embed_node = node_map[pattern_env['embed']]
  embed_node_module = utils.get_target_mod(traced,embed_node.target)
  

  
  
  class ReplacementClass(torch.nn.Module):
    def __init__(self):
      super().__init__()
      self.offsets = offsets_val
      self.redency_offset = torch.as_tensor(np.array(offsets_val[redency_part_slice[1]], dtype=np.int64))
      self.unredency_offset = torch.as_tensor(np.array(offsets_val[unredency_part_slice[1]], dtype=np.int64))
      embedding_config = embed_node_module.weight.data.shape
      self.embed = nn.Embedding(embedding_config[0],embedding_config[1])
      self.embed.weight.data.copy_(embed_node_module.weight.data)
    def forward(self,x):
      x = self.offsets  + x
      x = self.embed(x)
      square_of_sum = torch.sum(x, dim=1) ** 2
      
      
      
      redency_part_embed_pow = x[redency_part_slice]  ** 2
      unredency_part_embed_pow = x[unredency_part_slice] ** 2
      redency_part_embed_pow = redency_part_embed_pow.repeat(batch,1,1)
      embed = torch.concat([redency_part_embed_pow,unredency_part_embed_pow], dim = 1)
      sum_of_square = torch.sum(embed, dim = 1)
      ix = square_of_sum - sum_of_square
      ix = torch.sum(ix,dim = 1,keepdim=True)
      return 0.5 * ix
    
  
  return pattern,ReplacementClass(),_match

In [12]:
def workload_fm_pow(dim,redundancy):
  print(f"now gen workload of fm with config: dim: {dim}")
  fm_model_ori = fm.FactorizationMachineModel([100 for i in range(100)],dim)
  fm_model_traced_ori = symbolic_trace(fm_model_ori)
  
  fm_model_modify = fm.FactorizationMachineModel([100 for i in range(100)],dim)
  fm_model_traced_modify = symbolic_trace(fm_model_modify)
  pattern,replace,match = gen_pattern_replace_and_matcher_for_FM_pow(fm_model_traced_modify,
                                                                      (0,slice(None,redundancy,None)),(slice(None,None,None),slice(redundancy,None,None)),
                                                                      "linear_offsets")
  matches = subgraph_rewriter.replace_pattern_with_filters(fm_model_traced_modify, pattern, replace,[match])
  return fm_model_traced_ori,fm_model_traced_modify

In [13]:
ori , modify = workload_fm_pow(128,80)

now gen workload of fm with config: dim: 128


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node offsets target offsets offsets of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '


In [14]:
%timeit -n 1 -r 30 with torch.no_grad():ori(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))

55.4 ms ± 4.28 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [15]:
%timeit -n 1 -r 30 with torch.no_grad():modify(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))

78.5 ms ± 4.33 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [43]:
def gen_pattern_replace_and_matcher_for_FM_reduce(traced,
                                                  redency_part_slice,unredency_part_slice,
                                                  target_node_name,batch = 4096,match_func = None
                                                ):
  from torch.fx.passes.utils.matcher_utils import SubgraphMatcher

  class PatternClass(torch.nn.Module):
    def __init__(self):
      super().__init__()
      self.embed = torch.nn.Embedding(1, 1)
      # self.offsets = torch.as_tensor(np.array((0, *np.cumsum([10,10])[:-1]), dtype=np.int64))
      self.offsets = nn.Parameter(torch.as_tensor(np.array((0, *np.cumsum([10,10])[:-1]), dtype=np.int64)),False)
    def forward(self,x):
      x = x + self.offsets
      x = self.embed(x)
      square_of_sum = torch.sum(x, dim=1) ** 2
      sum_of_square = torch.sum(x ** 2, dim=1)
      ix = square_of_sum - sum_of_square    
      ix = torch.sum(ix, dim=1, keepdim=True)  
      return 0.5 * ix
  def _match(match,ori,pat):
    return True 
  # env  = utils.get_env(traced)
  # target_node = env[target_node_name]
  # target_node_mod = utils.get_target_mod(traced,target_node_name,"_")
  pattern = PatternClass()  
  pattern_trace = symbolic_trace(pattern)
  pattern_graph = pattern_trace.graph
  original_graph = traced.graph
  matcher =  SubgraphMatcher(pattern_graph, match_output=False, match_placeholder=False,
                              remove_overlapping_matches=True)
  _matches = matcher.match(original_graph)
  match_filters = [_match if match_func is None else match_func]
  _matches = [
      m for m in _matches
      if all(match_filter(m, original_graph, pattern_graph)
              for match_filter in match_filters)
  ]  
  # 因为在过滤器中做了限制应该只有一个符合要求的
  _matched = _matches[0]
  pattern_env = utils.get_env(pattern_trace)
  node_map = _matched.nodes_map
  pn = pattern_env['offsets']
  offsets_node = node_map[pn]
  offsets_val = utils.get_target_mod(traced,offsets_node.target)
  
  embed_node = node_map[pattern_env['embed']]
  embed_node_module = utils.get_target_mod(traced,embed_node.target)
  

  
  
  class ReplacementClass(torch.nn.Module):
    def __init__(self):
      super().__init__()
      self.offsets = offsets_val
      self.redency_offset = nn.Parameter(torch.as_tensor(np.array(offsets_val[redency_part_slice[1]], dtype=np.int64)),False)
      self.unredency_offset = nn.Parameter(torch.as_tensor(np.array(offsets_val[unredency_part_slice[1]], dtype=np.int64)),False)
      embedding_config = embed_node_module.weight.data.shape
      self.embed = nn.Embedding(embedding_config[0],embedding_config[1])
      self.embed.weight.data.copy_(embed_node_module.weight.data)
    def forward(self,x):
      x = self.offsets  + x
      x = self.embed(x)
      square_of_sum = torch.sum(x, dim=1) ** 2
      pow_x = x ** 2
      
      
      redency_part_embed_pow = pow_x[redency_part_slice] 
      unredency_part_embed_pow = pow_x[unredency_part_slice] 
      redency_embed_square_sum = torch.sum(redency_part_embed_pow ,dim=0)
      unredency_embed_square_sum = torch.sum(unredency_part_embed_pow ,dim=1)
      sum_of_square = redency_embed_square_sum + unredency_embed_square_sum
      # sum_of_square = torch.sum(embed, dim = 1)
      ix = square_of_sum - sum_of_square
      ix = torch.sum(ix,dim = 1,keepdim=True)
      return 0.5 * ix
    
  
  return pattern,ReplacementClass(),_match

In [44]:
def workload_fm_reduce(dim,redundancy):
  print(f"now gen workload of fm with config: dim: {dim}")
  fm_model_ori = fm.FactorizationMachineModel([100 for i in range(100)],dim)
  fm_model_traced_ori = symbolic_trace(fm_model_ori)
  
  fm_model_modify = fm.FactorizationMachineModel([100 for i in range(100)],dim)
  fm_model_traced_modify = symbolic_trace(fm_model_modify)
  pattern,replace,match = gen_pattern_replace_and_matcher_for_FM_reduce(fm_model_traced_modify,
                                                                      (0,slice(None,redundancy,None)),(slice(None,None,None),slice(redundancy,None,None)),
                                                                      "linear_offsets")
  matches = subgraph_rewriter.replace_pattern_with_filters(fm_model_traced_modify, pattern, replace,[match])
  return fm_model_traced_ori,fm_model_traced_modify

In [14]:
ori , modify = workload_fm_reduce(128,80)

now gen workload of fm with config: dim: 128


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node offsets target offsets offsets of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '


In [16]:
interp = utils.ProfilingInterpreter(ori)
interp.run(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))
print(interp.summary(True))

total true time 37.42265701293945 ms
total time: 54.779052734375 ms
Op type        Op                     Average runtime (ms)    Pct total runtime
-------------  -------------------  ----------------------  -------------------
call_module    embedding_embedding              16.974               30.9862
call_function  pow_2                            12.1913              22.2554
call_function  sum_2                             2.52962              4.61786
call_function  sum_3                             2.43092              4.43767
call_function  add                               0.763893             1.3945
call_module    linear_fc                         0.64826              1.18341
call_function  sub                               0.534773             0.976236
call_function  add_2                             0.53072              0.968837
call_function  pow_1                             0.224829             0.410428
call_function  sum_1                             0.128746             

In [15]:
interp = utils.ProfilingInterpreter(modify)
interp.run(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))
print(interp.summary(True))

total true time 37.03117370605469 ms
total time: 54.56137657165527 ms
Op type        Op                Average runtime (ms)    Pct total runtime
-------------  --------------  ----------------------  -------------------
call_module    embed                       16.7956              30.783
call_function  pow_2                       12.7232              23.3191
call_function  sum_2                        2.62594              4.81282
call_function  sum_4                        0.896215             1.64258
call_function  add                          0.795126             1.45731
call_module    linear_fc                    0.729084             1.33626
call_method    add_2                        0.55933              1.02514
call_function  add_3                        0.521421             0.95566
call_function  pow_1                        0.404358             0.741107
call_function  getitem                      0.16737              0.306755
call_function  sum_1                        0.13470

In [5]:
%timeit -n 1 -r 30 with torch.no_grad():ori(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))

63.1 ms ± 3.72 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [6]:
%timeit -n 1 -r 30 with torch.no_grad():modify(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))

61.6 ms ± 3.84 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [7]:
ori , modify = workload_fm_reduce(32,80)

now gen workload of fm with config: dim: 32


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node offsets target offsets offsets of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '


In [8]:
%timeit -n 1 -r 30 with torch.no_grad():ori(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))

23.9 ms ± 2.22 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [9]:
%timeit -n 1 -r 30 with torch.no_grad():modify(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))

23.7 ms ± 2.04 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [10]:
interp = utils.ProfilingInterpreter(ori)
interp.run(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))
print(interp.summary(True))

total true time 14.205694198608398 ms
total time: 20.94292640686035 ms
Op type        Op                     Average runtime (ms)    Pct total runtime
-------------  -------------------  ----------------------  -------------------
call_module    embedding_embedding               5.3072              25.3412
call_function  pow_2                             4.10414             19.5968
call_function  sum_3                             0.859499             4.10401
call_function  add                               0.828028             3.95373
call_function  sum_2                             0.745296             3.5587
call_module    linear_fc                         0.671864             3.20807
call_function  add_2                             0.581503             2.77661
call_function  sum_1                             0.267506             1.27731
call_function  pow_1                             0.18549              0.885691
call_function  sub                               0.157833            

In [11]:
interp = utils.ProfilingInterpreter(modify)
interp.run(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))
print(interp.summary(True))

total true time 14.345169067382812 ms
total time: 20.27416229248047 ms
Op type        Op                Average runtime (ms)    Pct total runtime
-------------  --------------  ----------------------  -------------------
call_module    embed                        5.16462             25.4739
call_function  pow_2                        4.32205             21.318
call_function  add                          0.828505             4.0865
call_function  sum_2                        0.712395             3.51381
call_module    linear_fc                    0.673771             3.3233
call_method    add_2                        0.626326             3.08928
call_function  add_3                        0.406504             2.00503
call_function  sum_4                        0.264406             1.30415
call_function  sub                          0.249386             1.23007
call_function  sum_5                        0.169516             0.836116
call_function  pow_1                        0.167131 

In [12]:
modify.print_readable()

class FactorizationMachineModel(torch.nn.Module):
    def forward(self, x):
        # No stacktrace found for following nodes
        linear_offsets = self.linear.offsets
        add = x + linear_offsets;  linear_offsets = None
        linear_fc = self.linear.fc(add);  add = None
        sum_1 = torch.sum(linear_fc, dim = 1);  linear_fc = None
        linear_bias = self.linear.bias
        add_1 = sum_1 + linear_bias;  sum_1 = linear_bias = None
        offsets = self.offsets
        add_4 = offsets.add(x);  offsets = x = None
        embed = self.embed(add_4);  add_4 = None
        sum_5 = torch.sum(embed, dim = 1)
        pow_3 = sum_5 ** 2;  sum_5 = None
        pow_4 = embed ** 2;  embed = None
        getitem = pow_4[(0, slice(None, 80, None))]
        getitem_1 = pow_4[(slice(None, None, None), slice(80, None, None))];  pow_4 = None
        sum_6 = torch.sum(getitem, dim = 0);  getitem = None
        sum_7 = torch.sum(getitem_1, dim = 1);  getitem_1 = None
        add_5 = sum_6 +

'class FactorizationMachineModel(torch.nn.Module):\n    def forward(self, x):\n        # No stacktrace found for following nodes\n        linear_offsets = self.linear.offsets\n        add = x + linear_offsets;  linear_offsets = None\n        linear_fc = self.linear.fc(add);  add = None\n        sum_1 = torch.sum(linear_fc, dim = 1);  linear_fc = None\n        linear_bias = self.linear.bias\n        add_1 = sum_1 + linear_bias;  sum_1 = linear_bias = None\n        offsets = self.offsets\n        add_4 = offsets.add(x);  offsets = x = None\n        embed = self.embed(add_4);  add_4 = None\n        sum_5 = torch.sum(embed, dim = 1)\n        pow_3 = sum_5 ** 2;  sum_5 = None\n        pow_4 = embed ** 2;  embed = None\n        getitem = pow_4[(0, slice(None, 80, None))]\n        getitem_1 = pow_4[(slice(None, None, None), slice(80, None, None))];  pow_4 = None\n        sum_6 = torch.sum(getitem, dim = 0);  getitem = None\n        sum_7 = torch.sum(getitem_1, dim = 1);  getitem_1 = None\n   

In [13]:
modify.graph.print_tabular()

opcode         name            target                                                      args                                                       kwargs
-------------  --------------  ----------------------------------------------------------  ---------------------------------------------------------  ---------------------------
placeholder    x               x                                                           ()                                                         {}
get_attr       linear_offsets  linear.offsets                                              ()                                                         {}
call_function  add             <built-in function add>                                     (x, linear_offsets)                                        {}
call_module    linear_fc       linear.fc                                                   (add,)                                                     {}
call_function  sum_1           <built-in method sum o

In [27]:
def gen_pattern_replace_and_matcher_for_FM_pow_reduce(traced,
                                                  redency_part_slice,unredency_part_slice,
                                                  target_node_name,batch = 4096,match_func = None
                                                ):
  from torch.fx.passes.utils.matcher_utils import SubgraphMatcher

  class PatternClass(torch.nn.Module):
    def __init__(self):
      super().__init__()
      self.embed = torch.nn.Embedding(1, 1)
      self.offsets = torch.as_tensor(np.array((0, *np.cumsum([10,10])[:-1]), dtype=np.int64))

    def forward(self,x):
      x = x + self.offsets
      x = self.embed(x)
      square_of_sum = torch.sum(x, dim=1) ** 2
      sum_of_square = torch.sum(x ** 2, dim=1)
      ix = square_of_sum - sum_of_square    
      ix = torch.sum(ix, dim=1, keepdim=True)  
      return 0.5 * ix
  def _match(match,ori,pat):
    return True 
  # env  = utils.get_env(traced)
  # target_node = env[target_node_name]
  # target_node_mod = utils.get_target_mod(traced,target_node_name,"_")
  pattern = PatternClass()  
  pattern_trace = symbolic_trace(pattern)
  pattern_graph = pattern_trace.graph
  original_graph = traced.graph
  matcher =  SubgraphMatcher(pattern_graph, match_output=False, match_placeholder=False,
                              remove_overlapping_matches=True)
  _matches = matcher.match(original_graph)
  match_filters = [_match if match_func is None else match_func]
  _matches = [
      m for m in _matches
      if all(match_filter(m, original_graph, pattern_graph)
              for match_filter in match_filters)
  ]  
  # 因为在过滤器中做了限制应该只有一个符合要求的
  _matched = _matches[0]
  pattern_env = utils.get_env(pattern_trace)
  node_map = _matched.nodes_map
  pn = pattern_env['offsets']
  offsets_node = node_map[pn]
  offsets_val = utils.get_target_mod(traced,offsets_node.target)
  
  embed_node = node_map[pattern_env['embed']]
  embed_node_module = utils.get_target_mod(traced,embed_node.target)
  

  
  
  class ReplacementClass(torch.nn.Module):
    def __init__(self):
      super().__init__()
      self.offsets = offsets_val
      self.redency_offset = torch.as_tensor(np.array(offsets_val[redency_part_slice[1]], dtype=np.int64))
      self.unredency_offset = torch.as_tensor(np.array(offsets_val[unredency_part_slice[1]], dtype=np.int64))
      embedding_config = embed_node_module.weight.data.shape
      self.embed = nn.Embedding(embedding_config[0],embedding_config[1])
      self.embed.weight.data.copy_(embed_node_module.weight.data)
    def forward(self,x):
      x = self.offsets  + x
      x = self.embed(x)
      square_of_sum = torch.sum(x, dim=1) ** 2
      
      
      
      redency_part_embed_pow = x[redency_part_slice]  ** 2
      unredency_part_embed_pow = x[unredency_part_slice] ** 2
      redency_embed_square_sum = torch.sum(redency_part_embed_pow ,dim=0)
      unredency_embed_square_sum = torch.sum(unredency_part_embed_pow ,dim=1)
      sum_of_square = redency_embed_square_sum + unredency_embed_square_sum
      ix = square_of_sum - sum_of_square
      ix = torch.sum(ix,dim = 1,keepdim=True)
      return 0.5 * ix
    
  
  return pattern,ReplacementClass(),_match

In [29]:
def workload_fm_reduce_pow(dim,redundancy):
  print(f"now gen workload of fm with config: dim: {dim}")
  fm_model_ori = fm.FactorizationMachineModel([100 for i in range(100)],dim)
  fm_model_traced_ori = symbolic_trace(fm_model_ori)
  
  fm_model_modify = fm.FactorizationMachineModel([100 for i in range(100)],dim)
  fm_model_traced_modify = symbolic_trace(fm_model_modify)
  pattern,replace,match = gen_pattern_replace_and_matcher_for_FM_pow_reduce(fm_model_traced_modify,
                                                                      (0,slice(None,redundancy,None)),(slice(None,None,None),slice(redundancy,None,None)),
                                                                      "linear_offsets")
  matches = subgraph_rewriter.replace_pattern_with_filters(fm_model_traced_modify, pattern, replace,[match])
  return fm_model_traced_ori,fm_model_traced_modify

In [30]:
ori , modify = workload_fm_reduce_pow(128,80)

now gen workload of fm with config: dim: 128


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node offsets target offsets offsets of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '


In [31]:
%timeit -n 1 -r 30 with torch.no_grad():ori(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))

52.8 ms ± 4.28 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [32]:
%timeit -n 1 -r 30 with torch.no_grad():modify(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))

36.3 ms ± 2.51 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [33]:
def gen_pattern_replace_and_matcher_for_FM_embed(traced,
                                                  redency_part_slice,unredency_part_slice,
                                                  target_node_name,batch = 4096,match_func = None
                                                ):
  from torch.fx.passes.utils.matcher_utils import SubgraphMatcher

  class PatternClass(torch.nn.Module):
    def __init__(self):
      super().__init__()
      self.embed = torch.nn.Embedding(1, 1)
      self.offsets = torch.as_tensor(np.array((0, *np.cumsum([10,10])[:-1]), dtype=np.int64))

    def forward(self,x):
      x = x + self.offsets
      x = self.embed(x)
      square_of_sum = torch.sum(x, dim=1) ** 2
      sum_of_square = torch.sum(x ** 2, dim=1)
      ix = square_of_sum - sum_of_square    
      ix = torch.sum(ix, dim=1, keepdim=True)  
      return 0.5 * ix
  def _match(match,ori,pat):
    return True 
  # env  = utils.get_env(traced)
  # target_node = env[target_node_name]
  # target_node_mod = utils.get_target_mod(traced,target_node_name,"_")
  pattern = PatternClass()  
  pattern_trace = symbolic_trace(pattern)
  pattern_graph = pattern_trace.graph
  original_graph = traced.graph
  matcher =  SubgraphMatcher(pattern_graph, match_output=False, match_placeholder=False,
                              remove_overlapping_matches=True)
  _matches = matcher.match(original_graph)
  match_filters = [_match if match_func is None else match_func]
  _matches = [
      m for m in _matches
      if all(match_filter(m, original_graph, pattern_graph)
              for match_filter in match_filters)
  ]  
  # 因为在过滤器中做了限制应该只有一个符合要求的
  _matched = _matches[0]
  pattern_env = utils.get_env(pattern_trace)
  node_map = _matched.nodes_map
  pn = pattern_env['offsets']
  offsets_node = node_map[pn]
  offsets_val = utils.get_target_mod(traced,offsets_node.target)
  
  embed_node = node_map[pattern_env['embed']]
  embed_node_module = utils.get_target_mod(traced,embed_node.target)
  

  
  
  class ReplacementClass(torch.nn.Module):
    def __init__(self):
      super().__init__()
      self.redency_offset = torch.as_tensor(np.array(offsets_val[redency_part_slice[1]], dtype=np.int64))
      self.unredency_offset = torch.as_tensor(np.array(offsets_val[unredency_part_slice[1]], dtype=np.int64))
      embedding_config = embed_node_module.weight.data.shape
      self.embed = nn.Embedding(embedding_config[0],embedding_config[1])
      self.embed.weight.data.copy_(embed_node_module.weight.data)
    def forward(self,x):
      redency_part = x[redency_part_slice] + self.redency_offset
      unredency_part = x[unredency_part_slice] + self.unredency_offset
      redency_embed = self.embed(redency_part)
      unredency_embed = self.embed(unredency_part)
      redency_embed = redency_embed.repeat(batch,1,1)
      embed = torch.concat([redency_embed, unredency_embed],dim = 1)
      square_of_sum = torch.sum(embed, dim=1) ** 2
      sum_of_square = torch.sum(embed ** 2,dim = 1)
      ix = square_of_sum - sum_of_square
      ix = torch.sum(ix,dim = 1,keepdim=True)
      return 0.5 * ix
    
  
  return pattern,ReplacementClass(),_match

In [34]:
def workload_fm_reduce_embed(dim,redundancy):
  print(f"now gen workload of fm with config: dim: {dim}")
  fm_model_ori = fm.FactorizationMachineModel([100 for i in range(100)],dim)
  fm_model_traced_ori = symbolic_trace(fm_model_ori)
  
  fm_model_modify = fm.FactorizationMachineModel([100 for i in range(100)],dim)
  fm_model_traced_modify = symbolic_trace(fm_model_modify)
  pattern,replace,match = gen_pattern_replace_and_matcher_for_FM_embed(fm_model_traced_modify,
                                                                      (0,slice(None,redundancy,None)),(slice(None,None,None),slice(redundancy,None,None)),
                                                                      "linear_offsets")
  matches = subgraph_rewriter.replace_pattern_with_filters(fm_model_traced_modify, pattern, replace,[match])
  return fm_model_traced_ori,fm_model_traced_modify

In [35]:
ori , modify = workload_fm_reduce_embed(128,80)

now gen workload of fm with config: dim: 128


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node redency_offset target redency_offset redency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node unredency_offset target unredency_offset unredency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '


In [36]:
%timeit -n 1 -r 30 with torch.no_grad():ori(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))

54.1 ms ± 3.59 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [37]:
%timeit -n 1 -r 30 with torch.no_grad():modify(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))

78.4 ms ± 4.87 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [37]:
def gen_pattern_replace_and_matcher_for_FM_reduce_stack(traced,
                                                  redency_part_slice,unredency_part_slice,
                                                  target_node_name,batch = 4096,match_func = None
                                                ):
  from torch.fx.passes.utils.matcher_utils import SubgraphMatcher

  class PatternClass(torch.nn.Module):
    def __init__(self):
      super().__init__()
      self.embed = torch.nn.Embedding(1, 1)
      self.offsets = torch.as_tensor(np.array((0, *np.cumsum([10,10])[:-1]), dtype=np.int64))

    def forward(self,x):
      x = x + self.offsets
      x = self.embed(x)
      square_of_sum = torch.sum(x, dim=1) ** 2
      sum_of_square = torch.sum(x ** 2, dim=1)
      ix = square_of_sum - sum_of_square    
      ix = torch.sum(ix, dim=1, keepdim=True)  
      return 0.5 * ix
  def _match(match,ori,pat):
    return True 
  # env  = utils.get_env(traced)
  # target_node = env[target_node_name]
  # target_node_mod = utils.get_target_mod(traced,target_node_name,"_")
  pattern = PatternClass()  
  pattern_trace = symbolic_trace(pattern)
  pattern_graph = pattern_trace.graph
  original_graph = traced.graph
  matcher =  SubgraphMatcher(pattern_graph, match_output=False, match_placeholder=False,
                              remove_overlapping_matches=True)
  _matches = matcher.match(original_graph)
  match_filters = [_match if match_func is None else match_func]
  _matches = [
      m for m in _matches
      if all(match_filter(m, original_graph, pattern_graph)
              for match_filter in match_filters)
  ]  
  # 因为在过滤器中做了限制应该只有一个符合要求的
  _matched = _matches[0]
  pattern_env = utils.get_env(pattern_trace)
  node_map = _matched.nodes_map
  pn = pattern_env['offsets']
  offsets_node = node_map[pn]
  offsets_val = utils.get_target_mod(traced,offsets_node.target)
  
  embed_node = node_map[pattern_env['embed']]
  embed_node_module = utils.get_target_mod(traced,embed_node.target)
  

  
  
  class ReplacementClass(torch.nn.Module):
    def __init__(self):
      super().__init__()
      self.offsets = offsets_val
      self.redency_offset = torch.as_tensor(np.array(offsets_val[redency_part_slice[1]], dtype=np.int64))
      self.unredency_offset = torch.as_tensor(np.array(offsets_val[unredency_part_slice[1]], dtype=np.int64))
      embedding_config = embed_node_module.weight.data.shape
      self.embed = nn.Embedding(embedding_config[0],embedding_config[1])
      self.embed.weight.data.copy_(embed_node_module.weight.data)
    def forward(self,x):
      x = self.offsets  + x
      x = self.embed(x)
      square_of_sum = torch.sum(x, dim=1) ** 2
      pow_x = x ** 2
      
      
      redency_part_embed_pow = pow_x[redency_part_slice] 
      unredency_part_embed_pow = pow_x[unredency_part_slice] 
      redency_embed_square_sum = torch.sum(redency_part_embed_pow ,dim=0)
      unredency_embed_square_sum = torch.sum(unredency_part_embed_pow ,dim=1)
      redency_embed_square_sum = redency_embed_square_sum.repeat(batch,1)
      sum_of_square = torch.stack([redency_embed_square_sum,unredency_embed_square_sum], dim = 1)
      sum_of_square = torch.sum(sum_of_square, dim = 1)
      # sum_of_square = torch.sum(embed, dim = 1)
      ix = square_of_sum - sum_of_square
      ix = torch.sum(ix,dim = 1,keepdim=True)
      return 0.5 * ix
    
  
  return pattern,ReplacementClass(),_match

In [38]:
def workload_fm_reduce_stack(dim,redundancy):
  print(f"now gen workload of fm with config: dim: {dim}")
  fm_model_ori = fm.FactorizationMachineModel([100 for i in range(100)],dim)
  fm_model_traced_ori = symbolic_trace(fm_model_ori)
  
  fm_model_modify = fm.FactorizationMachineModel([100 for i in range(100)],dim)
  fm_model_traced_modify = symbolic_trace(fm_model_modify)
  pattern,replace,match = gen_pattern_replace_and_matcher_for_FM_reduce_stack(fm_model_traced_modify,
                                                                      (0,slice(None,redundancy,None)),(slice(None,None,None),slice(redundancy,None,None)),
                                                                      "linear_offsets")
  matches = subgraph_rewriter.replace_pattern_with_filters(fm_model_traced_modify, pattern, replace,[match])
  return fm_model_traced_ori,fm_model_traced_modify

In [46]:
ori , modify = workload_fm_reduce(128,80)

now gen workload of fm with config: dim: 128


In [47]:
%timeit -n 1 -r 30 with torch.no_grad():ori(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))

61.3 ms ± 3.61 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [48]:
%timeit -n 1 -r 30 with torch.no_grad():modify(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))

59 ms ± 567 µs per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [49]:
ori.to(device)

FactorizationMachineModel(
  (linear): Module(
    (fc): Embedding(10000, 1)
  )
  (embedding): Module(
    (embedding): Embedding(10000, 128)
  )
)

In [56]:
x = torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long).to(device)
%timeit -n 1 -r 30 with torch.no_grad():ori(x)

214 µs ± 64 µs per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [51]:
modify.to(device)

FactorizationMachineModel(
  (linear): Module(
    (fc): Embedding(10000, 1)
  )
  (embed): Embedding(10000, 128)
)

In [60]:
x = torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long).to(device)
%timeit -n 1 -r 30 with torch.no_grad():modify(x)

297 µs ± 45.3 µs per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [48]:
ori , modify = workload_fm_reduce_stack(128,80)

now gen workload of fm with config: dim: 128


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node offsets target offsets offsets of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '


In [49]:
%timeit -n 1 -r 30 with torch.no_grad():ori(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))

66.1 ms ± 3.84 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [50]:
%timeit -n 1 -r 30 with torch.no_grad():modify(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))

67.6 ms ± 3.66 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [10]:
def gen_pattern_replace_and_matcher_for_FM_reduce_add(traced,
                                                  redency_part_slice,unredency_part_slice,
                                                  target_node_name,batch = 4096,match_func = None
                                                ):
  from torch.fx.passes.utils.matcher_utils import SubgraphMatcher

  class PatternClass(torch.nn.Module):
    def __init__(self):
      super().__init__()
      self.embed = torch.nn.Embedding(1, 1)
      # self.offsets = torch.as_tensor(np.array((0, *np.cumsum([10,10])[:-1]), dtype=np.int64))
      self.offsets = nn.Parameter(torch.as_tensor(np.array((0, *np.cumsum([10,10])[:-1]), dtype=np.int64)),False)

    def forward(self,x):
      x = x + self.offsets
      x = self.embed(x)
      square_of_sum = torch.sum(x, dim=1) ** 2
      sum_of_square = torch.sum(x ** 2, dim=1)
      ix = square_of_sum - sum_of_square    
      ix = torch.sum(ix, dim=1, keepdim=True)  
      return 0.5 * ix
  def _match(match,ori,pat):
    return True 
  # env  = utils.get_env(traced)
  # target_node = env[target_node_name]
  # target_node_mod = utils.get_target_mod(traced,target_node_name,"_")
  pattern = PatternClass()  
  pattern_trace = symbolic_trace(pattern)
  pattern_graph = pattern_trace.graph
  original_graph = traced.graph
  matcher =  SubgraphMatcher(pattern_graph, match_output=False, match_placeholder=False,
                              remove_overlapping_matches=True)
  _matches = matcher.match(original_graph)
  match_filters = [_match if match_func is None else match_func]
  _matches = [
      m for m in _matches
      if all(match_filter(m, original_graph, pattern_graph)
              for match_filter in match_filters)
  ]  
  # 因为在过滤器中做了限制应该只有一个符合要求的
  _matched = _matches[0]
  pattern_env = utils.get_env(pattern_trace)
  node_map = _matched.nodes_map
  pn = pattern_env['offsets']
  offsets_node = node_map[pn]
  offsets_val = utils.get_target_mod(traced,offsets_node.target)
  
  embed_node = node_map[pattern_env['embed']]
  embed_node_module = utils.get_target_mod(traced,embed_node.target)
  

  
  
  class ReplacementClass(torch.nn.Module):
    def __init__(self):
      super().__init__()
      self.offsets = offsets_val
      self.redency_offset = torch.as_tensor(np.array(offsets_val[redency_part_slice[1]], dtype=np.int64))
      self.unredency_offset = torch.as_tensor(np.array(offsets_val[unredency_part_slice[1]], dtype=np.int64))
      embedding_config = embed_node_module.weight.data.shape
      self.embed = nn.Embedding(embedding_config[0],embedding_config[1])
      self.embed.weight.data.copy_(embed_node_module.weight.data)
    def forward(self,x):
      x = self.offsets  + x
      x = self.embed(x)
      square_of_sum = torch.sum(x, dim=1) ** 2
      pow_x = x ** 2
      
      
      redency_part_embed_pow = pow_x[redency_part_slice] 
      unredency_part_embed_pow = pow_x[unredency_part_slice] 
      redency_embed_square_sum = torch.sum(redency_part_embed_pow ,dim=0)
      unredency_embed_square_sum = torch.sum(unredency_part_embed_pow ,dim=1)
      redency_embed_square_sum = redency_embed_square_sum.repeat(batch,1)
      sum_of_square = redency_embed_square_sum + unredency_embed_square_sum
      # sum_of_square = torch.sum(embed, dim = 1)
      ix = square_of_sum - sum_of_square
      ix = torch.sum(ix,dim = 1,keepdim=True)
      return 0.5 * ix
    
  
  return pattern,ReplacementClass(),_match

In [6]:
fm_model_ori = fm.FactorizationMachineModel([100 for i in range(100)],32)

In [7]:
fm_model_traced_ori = symbolic_trace(fm_model_ori)


In [8]:
fm_model_traced_ori.print_readable()

class FactorizationMachineModel(torch.nn.Module):
    def forward(self, x):
        # No stacktrace found for following nodes
        linear_offsets = self.linear.offsets
        add = x + linear_offsets;  linear_offsets = None
        linear_fc = self.linear.fc(add);  add = None
        sum_1 = torch.sum(linear_fc, dim = 1);  linear_fc = None
        linear_bias = self.linear.bias
        add_1 = sum_1 + linear_bias;  sum_1 = linear_bias = None
        embedding_offsets = self.embedding.offsets
        add_2 = x + embedding_offsets;  x = embedding_offsets = None
        embedding_embedding = self.embedding.embedding(add_2);  add_2 = None
        sum_2 = torch.sum(embedding_embedding, dim = 1)
        pow_1 = sum_2 ** 2;  sum_2 = None
        pow_2 = embedding_embedding ** 2;  embedding_embedding = None
        sum_3 = torch.sum(pow_2, dim = 1);  pow_2 = None
        sub = pow_1 - sum_3;  pow_1 = sum_3 = None
        sum_4 = torch.sum(sub, dim = 1, keepdim = True);  sub = None
        

'class FactorizationMachineModel(torch.nn.Module):\n    def forward(self, x):\n        # No stacktrace found for following nodes\n        linear_offsets = self.linear.offsets\n        add = x + linear_offsets;  linear_offsets = None\n        linear_fc = self.linear.fc(add);  add = None\n        sum_1 = torch.sum(linear_fc, dim = 1);  linear_fc = None\n        linear_bias = self.linear.bias\n        add_1 = sum_1 + linear_bias;  sum_1 = linear_bias = None\n        embedding_offsets = self.embedding.offsets\n        add_2 = x + embedding_offsets;  x = embedding_offsets = None\n        embedding_embedding = self.embedding.embedding(add_2);  add_2 = None\n        sum_2 = torch.sum(embedding_embedding, dim = 1)\n        pow_1 = sum_2 ** 2;  sum_2 = None\n        pow_2 = embedding_embedding ** 2;  embedding_embedding = None\n        sum_3 = torch.sum(pow_2, dim = 1);  pow_2 = None\n        sub = pow_1 - sum_3;  pow_1 = sum_3 = None\n        sum_4 = torch.sum(sub, dim = 1, keepdim = True);  s

In [11]:
def workload_fm_reduce_add(dim,redundancy):
  print(f"now gen workload of fm with config: dim: {dim}")
  fm_model_ori = fm.FactorizationMachineModel([100 for i in range(100)],dim)
  fm_model_traced_ori = symbolic_trace(fm_model_ori)
  
  fm_model_modify = fm.FactorizationMachineModel([100 for i in range(100)],dim)
  fm_model_traced_modify = symbolic_trace(fm_model_modify)
  pattern,replace,match = gen_pattern_replace_and_matcher_for_FM_reduce_add(fm_model_traced_modify,
                                                                      (0,slice(None,redundancy,None)),(slice(None,None,None),slice(redundancy,None,None)),
                                                                      "linear_offsets")
  matches = subgraph_rewriter.replace_pattern_with_filters(fm_model_traced_modify, pattern, replace,[match])
  return fm_model_traced_ori,fm_model_traced_modify

In [9]:
fm_model_traced_ori.graph.print_tabular()

opcode         name                 target                                                      args                      kwargs
-------------  -------------------  ----------------------------------------------------------  ------------------------  ---------------------------
placeholder    x                    x                                                           ()                        {}
get_attr       linear_offsets       linear.offsets                                              ()                        {}
call_function  add                  <built-in function add>                                     (x, linear_offsets)       {}
call_module    linear_fc            linear.fc                                                   (add,)                    {}
call_function  sum_1                <built-in method sum of type object at 0x7f17d3680760>      (linear_fc,)              {'dim': 1}
get_attr       linear_bias          linear.bias                                         

In [25]:
ori , modify = workload_fm_reduce_add(128,80)

now gen workload of fm with config: dim: 128


In [26]:
%timeit -n 1 -r 30 with torch.no_grad():ori(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))

61.2 ms ± 4.03 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [27]:
%timeit -n 1 -r 30 with torch.no_grad():modify(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))

59.5 ms ± 3.31 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [29]:
ori.to(device)

FactorizationMachineModel(
  (linear): Module(
    (fc): Embedding(10000, 1)
  )
  (embedding): Module(
    (embedding): Embedding(10000, 128)
  )
)

In [32]:
x = torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long).to(device)
%timeit -n 1 -r 30 with torch.no_grad():ori(x)

251 µs ± 71.8 µs per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [34]:
modify.to(device)

FactorizationMachineModel(
  (linear): Module(
    (fc): Embedding(10000, 1)
  )
  (embed): Embedding(10000, 128)
)

In [35]:
x = torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long).to(device)
%timeit -n 1 -r 30 with torch.no_grad():modify(x)

340 µs ± 96.9 µs per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [63]:
ori , modify = workload_fm(128,80)

now gen workload of fm with config: dim: 128


In [64]:
modify.to(device)

FactorizationMachineModel(
  (linear): Module(
    (fc): Embedding(10000, 1)
  )
  (embed): Embedding(10000, 128)
)

In [68]:
ori.to(device)

FactorizationMachineModel(
  (linear): Module(
    (fc): Embedding(10000, 1)
  )
  (embedding): Module(
    (embedding): Embedding(10000, 128)
  )
)

In [70]:
x = torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long).to(device)
%timeit -n 1 -r 30 with torch.no_grad():ori(x)

245 µs ± 71 µs per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [72]:
x = torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long).to(device)
%timeit -n 1 -r 30 with torch.no_grad():modify(x)

356 µs ± 75 µs per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [40]:
ori , modify = workload_fm(128,80)

now gen workload of fm with config: dim: 128


In [45]:
%timeit -n 1 -r 30 with torch.no_grad():ori(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))

63.3 ms ± 4.26 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [43]:
%timeit -n 1 -r 30 with torch.no_grad():modify(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))

22.1 ms ± 1.98 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [10]:
modify.to(device)
ori.to(device)

FactorizationMachineModel(
  (linear): Module(
    (fc): Embedding(10000, 1)
  )
  (embedding): Module(
    (embedding): Embedding(10000, 128)
  )
)

In [35]:
x = torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long).to(device)
%timeit -n 1 -r 30 with torch.no_grad():ori(x)

254 µs ± 41.4 µs per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [39]:
x = torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long).to(device)
%timeit -n 1 -r 30 with torch.no_grad():modify(x)

366 µs ± 63.6 µs per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [28]:
x = torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long).to(device)

interp = utils.ProfilingInterpreter(ori)
interp.run(x)
print(interp.summary(True))

total true time 0.8633136749267578 ms
total time: 1.0313987731933594 ms
Op type        Op                     Average runtime (ms)    Pct total runtime
-------------  -------------------  ----------------------  -------------------
call_module    linear_fc                        0.112295              10.8877
call_function  add                              0.0777245              7.53583
call_module    embedding_embedding              0.067234               6.51872
call_function  sum_1                            0.0574589              5.57097
call_function  pow_1                            0.0493526              4.78502
call_function  mul                              0.0472069              4.57698
call_function  sigmoid                          0.0460148              4.4614
call_function  sum_2                            0.0445843              4.3227
call_function  add_1                            0.0400543              3.8835
call_function  sub                              0.0374317    

In [31]:
x = torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long).to(device)

interp = utils.ProfilingInterpreter(modify)
interp.run(x)
print(interp.summary(True))

total true time 0.9818077087402344 ms
total time: 1.1548995971679688 ms
Op type        Op                  Average runtime (ms)    Pct total runtime
-------------  ----------------  ----------------------  -------------------
call_module    linear_fc                     0.102043               8.83567
call_function  add                           0.0693798              6.00743
call_module    embed                         0.0555515              4.81007
call_function  mul                           0.0510216              4.41784
call_function  sum_1                         0.0493526              4.27333
call_function  getitem                       0.0448227              3.88109
call_function  pow_1                         0.0412464              3.57143
call_module    embed_1                       0.0398159              3.44756
call_function  sum_2                         0.0352859              3.05533
call_function  add_1                         0.0312328              2.70438
call_function 

In [6]:
def reduceOp(x):
  return torch.sum(x, dim = 1)

In [7]:
class reduceRewriteOp(nn.Module):
    """
    A pytorch implementation of Factorization Machine.

    Reference:
        S Rendle, Factorization Machines, 2010.
    """

    def __init__(self,redundancy_slice,non_redundancy_slice):
        super().__init__()
        self.redundancy_slice = redundancy_slice
        self.non_redundancy_slice = non_redundancy_slice

    def forward(self, x):
      redundancy_emb = x[self.redundancy_slice]
      non_redundancy_emb = x[self.non_redundancy_slice]
      return torch.sum(redundancy_emb, dim = -2) + torch.sum(non_redundancy_emb, dim = -2)

In [113]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [138]:
ori = symbolic_trace(reduceOp)
modify = symbolic_trace(reduceRewriteOp((0,slice(None,80,None)),(slice(None,None,None),slice(80,None,None))))

In [144]:
x = torch.randn( size=(4096,100,128))
%timeit -n 1 -r 30 with torch.no_grad():ori(x)

4.03 ms ± 293 µs per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [114]:
ori.to(device)

reduceOp()

In [118]:
x = torch.randn( size=(4096,100,128)).to(device)
%timeit -n 1 -r 30 with torch.no_grad():ori(x)

The slowest run took 10.49 times longer than the fastest. This could mean that an intermediate result is being cached.
23.8 µs ± 28.6 µs per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [28]:
interp = utils.ProfilingInterpreter(ori)
interp.run(x)
print(interp.summary(True))

total true time 5.20014762878418 ms
total time: 5.33294677734375 ms
Op type        Op        Average runtime (ms)    Pct total runtime
-------------  ------  ----------------------  -------------------
call_function  sum_1                5.12409              96.0837
output         output               0.04673               0.876252
placeholder    x                    0.0293255             0.549893


In [119]:
modify.to(device)

reduceRewriteOp()

In [125]:
x = torch.randn( size=(4096,100,128)).to(device)
%timeit -n 1 -r 30 with torch.no_grad():modify(x)

The slowest run took 6.21 times longer than the fastest. This could mean that an intermediate result is being cached.
55.1 µs ± 42.2 µs per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [149]:
x = torch.randn( size=(4096,100,128))
%timeit -n 1 -r 30 with torch.no_grad():modify(x)

The slowest run took 5.44 times longer than the fastest. This could mean that an intermediate result is being cached.
381 µs ± 269 µs per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [102]:
interp = utils.ProfilingInterpreter(modify)
interp.run(x)
print(interp.summary(True))

total true time 1.5444755554199219 ms
total time: 1.789093017578125 ms
Op type        Op           Average runtime (ms)    Pct total runtime
-------------  ---------  ----------------------  -------------------
call_function  sum_2                   0.804663              44.976
call_function  add                     0.380993              21.2953
call_function  getitem                 0.128031               7.15618
call_function  sum_1                   0.0972748              5.4371
call_function  getitem_1               0.0579357              3.23827
placeholder    x                       0.0431538              2.41205
output         output                  0.0324249              1.81237


In [61]:
class reduceRewriteOpComplex1(nn.Module):
    """
    A pytorch implementation of Factorization Machine.

    Reference:
        S Rendle, Factorization Machines, 2010.
    """

    def __init__(self,redundancy_slice,non_redundancy_slice,batch=4096):
        super().__init__()
        self.redundancy_slice = redundancy_slice
        self.non_redundancy_slice = non_redundancy_slice
        self.batch = batch

    def forward(self, x):
      redundancy_emb = x[self.redundancy_slice]
      non_redundancy_emb = x[self.non_redundancy_slice]
      redundancy_emb = torch.sum(redundancy_emb, dim = -2)
      redundancy_emb = redundancy_emb.repeat(self.batch, 1)
      return redundancy_emb + torch.sum(non_redundancy_emb, dim = -2)

In [150]:
modify1 = symbolic_trace(reduceRewriteOpComplex1((0,slice(None,80,None)),(slice(None,None,None),slice(80,None,None))))

In [126]:
modify1.to(device)

reduceRewriteOpComplex1()

In [130]:
x = torch.randn( size=(4096,100,128)).to(device)
%timeit -n 1 -r 30 with torch.no_grad():modify1(x)

The slowest run took 5.59 times longer than the fastest. This could mean that an intermediate result is being cached.
74.6 µs ± 51 µs per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [154]:
x = torch.randn( size=(4096,100,128))
%timeit -n 1 -r 30 with torch.no_grad():modify1(x)

The slowest run took 6.27 times longer than the fastest. This could mean that an intermediate result is being cached.
428 µs ± 285 µs per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [105]:
interp = utils.ProfilingInterpreter(modify1)
interp.run(x)
print(interp.summary(True))

total true time 1.1098384857177734 ms
total time: 1.2545585632324219 ms
Op type        Op           Average runtime (ms)    Pct total runtime
-------------  ---------  ----------------------  -------------------
call_function  sum_2                   0.523806             41.7522
call_method    repeat                  0.275135             21.9308
call_function  add                     0.145912             11.6306
call_function  getitem                 0.0641346             5.11212
call_function  sum_1                   0.0445843             3.55378
placeholder    x                       0.0264645             2.10946
call_function  getitem_1               0.0190735             1.52033
output         output                  0.0107288             0.855188


In [106]:
class reduceRewriteOpComplex2(nn.Module):
    """
    A pytorch implementation of Factorization Machine.

    Reference:
        S Rendle, Factorization Machines, 2010.
    """

    def __init__(self,redundancy_slice,non_redundancy_slice,batch=4096):
        super().__init__()
        self.redundancy_slice = redundancy_slice
        self.non_redundancy_slice = non_redundancy_slice
        self.batch = batch

    def forward(self, x):
      redundancy_emb = x[self.redundancy_slice]
      non_redundancy_emb = x[self.non_redundancy_slice]
      redundancy_emb = torch.sum(redundancy_emb,dim = -2)
      non_redundancy_emb = torch.sum(non_redundancy_emb,dim = -2)
      redundancy_emb = redundancy_emb.repeat(self.batch, 1)
      emb = torch.stack([redundancy_emb,non_redundancy_emb], dim = 1)
      return torch.sum(emb, dim = -2)

In [155]:
modify2 = symbolic_trace(reduceRewriteOpComplex2((0,slice(None,80,None)),(slice(None,None,None),slice(80,None,None))))

In [ ]:
modify2.to(device)

reduceRewriteOpComplex2()

In [136]:
x = torch.randn( size=(4096,100,128)).to(device)
%timeit -n 1 -r 30 with torch.no_grad():modify2(x)

The slowest run took 4.74 times longer than the fastest. This could mean that an intermediate result is being cached.
91.4 µs ± 51.3 µs per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [158]:
x = torch.randn( size=(4096,100,128))
%timeit -n 1 -r 30 with torch.no_grad():modify2(x)

The slowest run took 5.34 times longer than the fastest. This could mean that an intermediate result is being cached.
671 µs ± 391 µs per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [112]:
interp = utils.ProfilingInterpreter(modify2)
interp.run(x)
print(interp.summary(True))

total true time 2.0186901092529297 ms
total time: 4.133939743041992 ms
Op type        Op           Average runtime (ms)    Pct total runtime
-------------  ---------  ----------------------  -------------------
call_function  sum_2                   0.939608             22.7291
call_function  stack                   0.309229              7.48025
call_function  sum_3                   0.272751              6.59784
call_method    repeat                  0.183582              4.44086
call_function  getitem                 0.113487              2.74526
call_function  sum_1                   0.0874996             2.11662
call_function  getitem_1               0.0560284             1.35533
placeholder    x                       0.0422001             1.02082
output         output                  0.0143051             0.346041
